In [82]:
import os
import numpy as np
import pandas as pd
import psycopg2

In [83]:
ls

 Volume in drive D has no label.
 Volume Serial Number is 0C45-8BC1

 Directory of d:\Coding\Git\PosgresIngco

2022-08-26  17:09    <DIR>          .
2022-08-26  17:09    <DIR>          ..
2022-08-26  14:47             3,131 2022 Revenue 26aug$.csv
2022-08-26  14:47             3,131 pen jualan$.csv
2022-08-26  17:14             3,132 pen_jualan.csv
2022-08-19  11:22                34 README.md
2022-08-26  13:29             7,463 test.ipynb
               5 File(s)         16,891 bytes
               2 Dir(s)  56,684,576,768 bytes free


In [84]:
df = pd.read_csv("pen jualan$.csv", parse_dates=["Tanggal"])
df.head()

,Tanggal,Sku,No. Pesanan,QTY,Asal Gudang,Total,Sumber,Jenis_Penjualan,Kode Sumber
0,2020-10-13,CAGLI1001,INV/20200928/XX/IX/639426010,1,HQ [Online] - Bintaro,"Rp800,000",Tokopedia,Online,4
1,2020-10-13,HKISSD12101,INV/20201013/XX/X/650891804,1,TC Jakarta - TokoCabang,"Rp221,500",Tokopedia,Online,4
2,2020-10-13,ID6808,INV/20201013/XX/X/651141057,1,TC Jakarta - TokoCabang,"Rp384,413",Tokopedia,Online,4
3,2020-10-14,HKNS16518,INV/20201014/XX/X/651431074,1,TC Jakarta - TokoCabang,"Rp15,500",Tokopedia,Online,4
4,2020-10-14,HKSD0628,INV/20201014/XX/X/651431074,1,TC Jakarta - TokoCabang,"Rp71,500",Tokopedia,Online,4


In [85]:
#rename csv, force lower case, no spaces, no dashes
file = "pen jualan$"
clean_tbl_name = file.lower().replace(" ", "_").replace("-","_").replace(r"/","_").replace("\\","_").replace("$","").replace("%","")

clean_tbl_name

'pen_jualan'

In [86]:
df.columns = [x.lower().replace(" ", "_").replace("-","_").replace(r"/","_").replace("\\","_").replace("$","").replace("%","").replace(".", "") for x in df.columns]

df.columns

Index(['tanggal', 'sku', 'no_pesanan', 'qty', 'asal_gudang', 'total', 'sumber',
       'jenis_penjualan', 'kode_sumber'],
      dtype='object')

In [87]:
df.dtypes

tanggal            datetime64[ns]
sku                        object
no_pesanan                 object
qty                         int64
asal_gudang                object
total                      object
sumber                     object
jenis_penjualan            object
kode_sumber                 int64
dtype: object

In [88]:
replacements = {'timedelta64[ns]': 'varchar',
        'object': 'varchar',
        'float64': 'float',
        'int64': 'int',
        'datetime64': 'timestamp',
        'datetime64[ns]': 'timestamp'}

col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(replacements)))
col_str

'tanggal timestamp, sku varchar, no_pesanan varchar, qty int, asal_gudang varchar, total varchar, sumber varchar, jenis_penjualan varchar, kode_sumber int'

In [89]:
conn_string = "host= zubdb.cabyoqiwyaa1.us-west-1.rds.amazonaws.com dbname= postgres user= zubdev password= E3413o079#" 
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()
print('opened database successfully')

opened database successfully


In [90]:
#drop table with the same name
cursor.execute("drop table if exists pen_jualan;")
print('sukses menghapus yang sebelumnya')

sukses menghapus yang sebelumnya


In [91]:
#create table
cursor.execute("create table pen_jualan (tanggal timestamp, sku varchar, no_pesanan varchar, qty int, asal_gudang varchar, total varchar, sumber varchar, jenis_penjualan varchar, kode_sumber int);")
print('table terbuat')

table terbuat


In [92]:
#insert value to table 

## save df to csv
df.to_csv('pen_jualan.csv', header=df.columns, index=False, encoding='utf-8')

##open the csv file, save it as an object and upload to db
my_file = open('pen_jualan.csv')
print("file dibuka di memory")

file dibuka di memory


In [93]:
#upload to db
SQL_STATEMENT = """
    COPY pen_jualan FROM STDIN WITH
        CSV
        HEADER
        DELIMITER AS ','
    """

cursor.copy_expert(sql=SQL_STATEMENT, file=my_file)
print('file copied to db')

file copied to db


In [94]:
cursor.execute("grant select on table pen_jualan to public")
conn.commit()
cursor.close()
print('table penjualan imported to db completed')

table penjualan imported to db completed


In [95]:
## Oke zub ini sudah sukses masuk, kursor terakhir buat select schema ternyata, ini aku belom nyobain kalau dia ke rawinput gimana
# 